# Animator

## Setting up the Simulation Environment

Import necessary `condynsate` modules. Also import `numpy` for array management. 

In [1]:
from condynsate import Simulator as conSim
from condynsate import __assets__ as assets
import numpy as np

Create a instance of `condynsate.simulator` that does not use the keyboard but does use the animator and the visualizer.

In [2]:
simulator = conSim(keyboard = False,
                   visualization = True,
                   visualization_fr = 24.0,
                   animation = True,
                   animation_fr = 12.0)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


Load three large planes into the simulator to represent the ground and walls. Set `fixed = True` so that they are fixed in space, and set `update_vis = False`. Because the ground and walls will not move, there is no need to send updates to the visualizer. This saves resources. Further, adjust the position and rotation of the walls so they look good.

In [3]:
# Load the ground
ground = simulator.load_urdf(urdf_path = assets['plane_big'],
                             fixed = True,
                             update_vis = False)

# Load the walls
left_wall = simulator.load_urdf(urdf_path = assets['plane_big'],
                                tex_path = assets['concrete_img'], # Choose a default texture for appearance
                                position = [0., -5., 0.],
                                roll = -1.5707,
                                fixed = True,
                                update_vis = False)
right_wall = simulator.load_urdf(urdf_path = assets['plane_big'],
                                 tex_path = assets['concrete_img'],  # Choose a default texture for appearance
                                 position = [0., 5., 0.],
                                 roll = 1.5707,
                                 fixed = True,
                                 update_vis = False)

Load the cart into the simulator. Set `fixed = False` so that the cart is able to freely move. Set `update_vis = True` so that its position updates are reflected in the visualizer. Additionally, set the position to `[0.0, 0.0, 0.25]` so that its wheels rest on the ground plane, and set yaw to 90 degrees so that is moves in the Y plane.

In [4]:
cart = simulator.load_urdf(urdf_path = assets['cart'],
                           position = [0., 0., 0.25],
                           yaw = 1.5707,
                           fixed = False,
                           update_vis = True)

Set the each wheel joint's and the pendulum arm joint's friction to some small value.

In [5]:
# Set joint damping
joint_names = ('chassis_to_wheel_1',
               'chassis_to_wheel_2',
               'chassis_to_wheel_3',
               'chassis_to_wheel_4',
               'chassis_to_arm',)
for joint_name in joint_names:
    simulator.set_joint_damping(urdf_obj = cart,
                               joint_name = joint_name,
                               damping = 0.01)

Set the initial angle of the pendulum. We denote that this is an initial condition with the `initial_cond = True` flag and we make the change instantaneously by using the `physics = False` flag.

In [6]:
initial_angle = 0.1745
simulator.set_joint_position(urdf_obj = cart,
                             joint_name = 'chassis_to_arm',
                             position = initial_angle, # For a rotational joint, this value is in radians
                             initial_cond = True,
                             physics = False)

## Setting up the Animator

Because we set `animator = True` in the simulator initialization, the animator is ready to add new subplots to it. To do this we use the `condynsate.Simulator.add_subplot` function.

In the first subplot, we want to plot only the wheel torque as a function of time. To do so, we create a line plot with 1 artist. This is done with the arguments `n_artists = 1` and `subplot_type = 'line'`. Further wee add a subplot title and axis labels with the `title = "Torque vs Time"`, `x_label = "Time [Seconds]"`, and `y_label = "Torque [Nm]"`, respectively. Next, to format the line itself, we set its color to black with `colors = ["k"]`, set its line width to 2.5 with `line_widths = [2.5]`, and make it a solid line with `line_styles = ["-"]`. Finally, we limit the y-axis view between -0.8 and 0.8 with `y_lim = [-0.80, 0.80]` and add a thin horizontal line across y=0 with `h_zero_line = True`.

In [7]:
subplot_1, suplot_1_artists = simulator.add_subplot(n_artists = 1,
                                                    subplot_type = 'line',
                                                    title = "Torque vs Time",
                                                    x_label = "Time [Seconds]",
                                                    y_label = "Torque [Nm]",
                                                    colors = ["k"],      # List of length equal to n_artists
                                                    line_widths = [2.5], # List of length equal to n_artists
                                                    line_styles = ["-"], # List of length equal to n_artists
                                                    y_lim = [-0.80, 0.80],
                                                    h_zero_line = True)

We make a similar subplot for the pendulum and wheel angles. Because we want two lines on this subplot, we call for 2 artists, one for each line. All other arguments are similar aside for `labels = ["Pendulum", "Wheel"]` which tells the animator what each arist should be called for the legend. 

In [8]:
subplot_2, suplot_2_artists = simulator.add_subplot(n_artists = 2,
                                                    subplot_type = 'line',
                                                    title = "Angles vs Time",
                                                    x_label = "Time [Seconds]",
                                                    y_label = "Angles [Rad]",
                                                    colors = ["m", "c"],      # List of length equal to n_artists
                                                    line_widths = [2.5, 2.5], # List of length equal to n_artists
                                                    line_styles = ["-", "-"], # List of length equal to n_artists
                                                    labels = ["Pendulum", "Wheel"],
                                                    h_zero_line = True)

The subplots are now set up how we want, so we can open the animator GUI. Once opened, no additional subplots can be added.

In [9]:
simulator.open_animator_gui()

As expected, a figure with our two subplots opens.

## Running the Simulation

Run the simulation in a loop. Begin by resetting the simulation using `condynsate.Simulator.reset` to reset the simulation to an initial state. Then we create a simulation loop. The loop terminates when the flag `condynsate.Simulator.is_done` flips to True. In each time step of the simulation loop we do four things:
1. Take a simulation time step.
2. If a successful simulation step was taken, read the wheel angles, wheel angular velocities, pendulum angle, and pendulum angular velocity. Successful simulation steps are indicated by `condynsate.Simulator.step` having a `ret_code` greater than `0`.
3. If a successful simulation step was taken, apply torques to wheels based on the mean wheel angle, mean wheel angular velocity, pendulum angle, and pendulum angular velocity.
4. If a successful simulation step was taken, update the animator.

Let's start by making a function that reads each angle and angular velocity we want.

In [10]:
def read_states(simulator, cart):
    # Define which joints to read
    joint_names = ('chassis_to_wheel_1',
                   'chassis_to_wheel_2',
                   'chassis_to_wheel_3',
                   'chassis_to_wheel_4',
                   'chassis_to_arm',)

    # Loop over each joint and read the state extracting angle and angular vel.
    angles = []
    angular_vels = []
    for joint_name in joint_names:
        state = simulator.get_joint_state(urdf_obj = cart,
                                          joint_name = joint_name)
        angles.append(state['position'])
        angular_vels.append(state['velocity'])

    # For the wheels, return the mean angle and angular vel.
    angles = [np.mean(angles[:-1]), angles[-1]]
    angular_vels = [np.mean(angular_vels[:-1]), angular_vels[-1]]
    return (angles[0], angular_vels[0]), (angles[1], angular_vels[1])

Next we make a function that applies torque based on the state we read.

In [11]:
def apply_torque(simulator, cart, mean_wheel_state, pendulum_state):
    # Calculate the torque via magic
    K = np.array([[ -2. ,  -0.1, -10. ,  -0.1]])
    m_e = np.zeros((4,1))
    n_e = np.zeros((1,1))
    m = np.array([[pendulum_state[1]],
                  [mean_wheel_state[1]],
                  [pendulum_state[0]],
                  [mean_wheel_state[0]]])
    torque = (-K@(m - m_e) + n_e).flatten()[0]
    torque = np.clip(torque, -0.75, 0.75)
    
    # Define which joints we apply torque to
    joint_names = ('chassis_to_wheel_1',
                   'chassis_to_wheel_2',
                   'chassis_to_wheel_3',
                   'chassis_to_wheel_4',)

    # Apply the torque we calculated
    for joint_name in joint_names:
        simulator.set_joint_torque(urdf_obj = cart,
                                   joint_name = joint_name,
                                   torque = torque,
                                   show_arrow = True, # Define arrow params to visualize the torque
                                   arrow_scale = 0.25,
                                   arrow_offset = 0.025)

    return torque

Next we want to build a function that updates the first subplot. We want this function to add a single datum to the end of the data list being plotted. To do this, we will use the `condynsate.Simulator.add_subplot_point` function. The arguments to this function are the subplot to which the point is added, the artist index which draws the point, and the (x,y) coordinates of the point. In our case, the subplot is called `subplot_1` and it only has one artist, `suplot_1_artists[0]`. The x coordinate will be simulation time which we extract with `condynsate.Simulator.time`, and the y coordinate is the torque we calculated in the previous function. After `condynsate.Simulator.add_subplot_point` is called, the animator automatically updates to add the point at the specified frame rate.

In [12]:
def add_torque_point(simulator, subplot, suplot_artists, torque):
    simulator.add_subplot_point(subplot_index = subplot,
                                artist_index = suplot_artists[0], # artists is always list but add_subplot_point
                                x = simulator.time,               # expects int. Therefore, index suplot_artists.
                                y = torque)

We build a similar function for the wheel and pendulum angles, but this time we interact with two artists on the same subplot by calling `condynsate.Simulator.add_subplot_point` twice, once for each artist.

In [13]:
def add_state_points(simulator, subplot, suplot_artists, mean_wheel_state, pendulum_state):
    simulator.add_subplot_point(subplot_index = subplot,
                                artist_index = suplot_artists[0], # artists is always list but add_subplot_point
                                x = simulator.time,               # expects int. Therefore, index suplot_artists.
                                y = pendulum_state[0])
    simulator.add_subplot_point(subplot_index = subplot,
                                artist_index = suplot_artists[1], # artists is always list but add_subplot_point
                                x = simulator.time,               # expects int. Therefore, index suplot_artists.
                                y = mean_wheel_state[0])

Finally, we build our simulation loop. By setting the `real_time = True` flag and `max_time = 10.0` in `condynsate.Simulator.step`, we tell the simulation to attempt to run in real time for 5 seconds.

In [14]:
# Reset before running a simulation loop
simulator.reset()
    
# Run the simulation loop
while(not simulator.is_done):
    # Step the sim
    ret_code = simulator.step(real_time = True,
                              max_time = 10.0)

    # If successful step was taken
    if ret_code > 0:
        # Read the states and apply the torque
        mean_wheel_state, pendulum_state = read_states(simulator, cart)
        torque = apply_torque(simulator, cart, mean_wheel_state, pendulum_state)

        # Add the new data points to the animator
        add_torque_point(simulator, subplot_1, suplot_1_artists, torque)
        add_state_points(simulator, subplot_2, suplot_2_artists, mean_wheel_state, pendulum_state)
        

Watch as the pendulum magically balances while the torque and angles are plotted in real time on the animator GUI.